In [1]:
import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import gradio as gr

In [22]:
# Set environment variables
# Initialize model

import os
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ["AZURE_OPENAI_ENDPOINT_SCUS"]
os.environ["AZURE_OPENAI_API_KEY"]  = os.environ["AZURE_OPENAI_API_KEY_SCUS"]
os.environ["OPENAI_API_VERSION"]    = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["AZURE_OPENAI_API_TYPE"] = os.environ["OPENAI_API_TYPE"]

MODEL = os.environ["GPT4-0125PREVIEW-128k"]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Research Agents"
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_KEY"]

In [23]:
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=1000)

In [24]:
# 1. Define custom tools
@tool("internet_search", return_direct=False)
def internet_search(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search, process_content]

In [25]:
# 2. Agents 
# Helper function for creating agents
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

# Define agent nodes
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


In [26]:
[element["title"] for element in internet_search("What is a CLP?")]

["Commercial Learner's Permit - How To Get a CLP - CDLTraining.org",
 '7 Best Gun Oil, CLP, and Grease [Tested] - Pew Pew Tactical',
 'CLP Prep Guide: How to Study for Your Permit Test',
 "What is a CLP? | How to Get a Commercial Learner's Permit | ACV Auctions",
 "What is a Commercial Learner's Permit and how do you get one?"]